## Submission by Johannes Kohlmann, Veronika Mayerhofer

In [2]:
import numpy as np
import pathpy as pp
import math

In [3]:
net = pp.Network(directed=False)
net.add_edges(("a","b"), ("a","c"), ("b","c"), ("b","d"), ("d","e"), ("e","f"), ("f", "g"), ("f", "d"), ("g", "d"))
for e in net.edges:
    e.we = 1

## Implementation of the Bellman-Ford Algorithm

In [4]:
def bellman_ford(net: pp.Network, source: pp.Node):
    # Initialize distances to infinity and predecessors to None
    for v in net.nodes:
        v.dist = math.inf
        v.predecessor = None

    # Distance from source to itself is zero
    source.dist = 0

    # Loop until "convergence"
    for _ in range(len(net.nodes) - 1):
        for e in net.edges:
            u,v,w = *tuple(e.nodes), e.we
            if u.dist + w < v.dist:
                v.dist = u.dist + w
                v.predecessor = u
            
            # If the network is not directed we have to consider the "reverse edeges"
            if not net.directed:
                if v.dist + w < u.dist:
                    u.dist = v.dist + w
                    u.predecessor = v

    for e in net.edges:
        u,v,w = *tuple(e.nodes), e.we
        if u.dist + w < v.dist or not net.directed and v.dist + w < u.dist:
            raise Exception("Network contains negative cycles!") 

In [5]:
bellman_ford(net, net.nodes["a"])

In [6]:
for node in net.nodes:
    predecessor = node.predecessor.uid if node.predecessor is not None else None
    print(f"{node.uid}: Dist = {node.dist}, Predecessor = {predecessor}")

a: Dist = 0, Predecessor = None
b: Dist = 1, Predecessor = a
c: Dist = 1, Predecessor = a
d: Dist = 2, Predecessor = b
e: Dist = 3, Predecessor = d
f: Dist = 3, Predecessor = d
g: Dist = 3, Predecessor = d


## Computing the diameter of a graph via adjacency matrix powers

In [7]:

net = pp.Network(directed=True)
net.add_edges(("a","b"), ("a","c"), ("b","c"), ("b","d"), ("d","e"), ("e","f"), ("f", "g"), ("f", "d"), ("g", "d"))

In [8]:
def diameter(net: pp.Network) -> int:
   
   A = net.adjacency_matrix()
   n = net.number_of_nodes()

   # The k-th power of A
   Ak = A
   k = 1

   # S[i,j] holds the shortest-path length from i to j
   S = np.full(fill_value=np.inf, shape=Ak.shape)
   np.fill_diagonal(S, 0)

   # Tracks how many shortest-paths we have found already
   num_updates = 0

   # We can terminate when:
   #  - We have found n**2 - n shortest paths (i.e. all of them that are not a cycle from node a node to itself)
   #  - We have reached k = n since no shortest path in a graph with n nodes has length > n - 1 if it exists
   while num_updates < n**2 - n and k < n:
      # x and y hold the "coordinates" of the nonzero entries of A**k. These are interesting since they might include a new shortest path
      x,y = Ak.nonzero()
      for i,j in zip(x,y):
         # If we have not found a shortest-path from i to j yet, we update S accordingly.
         if S[i,j] == np.inf:
            S[i,j] = k
            num_updates = num_updates + 1
      
      # We have discovered all shortest paths of lenght k and can move on to k+1 now.
      Ak = Ak * A 
      k = k + 1
   
   # Since the diameter is defined as the longest shortest path, we return the maximum of all of those we have found (or infinity if atleast two nodes are not connected).
   return S.max()
       

## Network where the node with the largest betweenes centrality has the smallest degree centrality

In [9]:
net = pp.Network(directed=False)
for i in range(2):
    for j in range(4):
        net.add_node(f"{i}_{j}")
        for k in range(j):
            net.add_edge(f"{i}_{j}",f"{i}_{k}")

net.add_node("connector")
net.add_edges(("0_0","connector"), ("1_0","connector"))

In [10]:
def betweenness_centrality(network: pp.Network, normalized: bool = False):
    from collections import defaultdict
    all_paths = pp.algorithms.all_shortest_paths(network, weight=False, return_distance_matrix=False)
    bw: defaultdict = defaultdict(float)

    for s in all_paths:
        for t in all_paths[s]:

            # number of shortest paths
            N_st = len(all_paths[s][t])

            # for each shortest path
            for p in all_paths[s][t]:

                # for each node other than the start and end
                for x in p[1:-1]:
                    if s != t != x:
                        bw[x] += 1.0 / N_st

    # assign zero values to nodes not occurring on shortest paths
    for v in network.nodes.uids:
        bw[v] += 0

    # normalize to ensure centralities are in range [0,1]
    if normalized:
        max_centr = max(bw.values())
        min_centr = min(bw.values())
        for v in bw:
            bw[v] = (bw[v] - min_centr) / (max_centr - min_centr)
    return bw

In [11]:
betweenness_centralities = betweenness_centrality(net)
degree_centralities = net.degrees()

In [12]:
assert betweenness_centralities["connector"] == max(betweenness_centralities.values())
assert degree_centralities["connector"] == min(degree_centralities.values())